In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=CU1c2__W9X2QPm9LQiFM1zRNMqzNWdCUC2SmWL8h3BY&tc=UMwBaiqq4S4tJ7fxhKokheBpqR7CAwvtZWdTD6MyCmI&cc=jw0bb0hbBEmZz9Mi-FCopCCfdqvEikXi6KS4iBkHwRs

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qhtGMcKzxnpHdAiM3n_uxN6H4KCVQ9-tD2DwpTWw0RpTONGbEM2kxc

Successfully saved authorization token.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
dataset = ee.FeatureCollection("users/zahraghahremani/Imagary_site")
siteID = dataset.aggregate_array('site_no');
lower = 4300
upper = 4600
siteID1 = siteID.slice(lower, upper)

folderDEM = 'DEM_SI_'+str(lower)+'_'+str(upper)
folderNAIP = 'NAIP_SI_'+str(lower)+'_'+str(upper)
os.mkdir('/content/gdrive/MyDrive/' + folderDEM)
os.mkdir('/content/gdrive/MyDrive/' + folderNAIP)
len(siteID1.getInfo())

300

In [ ]:
from ee import feature
from pprint import pprint
import time 
import folium
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
ID_errors = []
for id in siteID1.getInfo():
  feature = dataset.filterMetadata('site_no', 'equals', id)#ee.Feature(dataset.filter(ee.Filter.eq('site_no', str(id))))
  # geometry = ee.Feature(featureList.get(j))
  # site_ID = geometry.get('site_no')
  coordinate = ee.List(feature.geometry().coordinates())
  buffer = ee.Number(0.0025)
  xMin = ee.Number(coordinate.get(0)).subtract(buffer)
  yMin = ee.Number(coordinate.get(1)).subtract(buffer) 
  xMax = ee.Number(coordinate.get(0)).add(buffer)
  yMax = ee.Number(coordinate.get(1)).add(buffer)
  
  myPolygon = ee.Geometry.Rectangle(
    [
      [xMin, yMin],
      [xMax, yMax]  
    ]
  )
 
  
# #   // ____________________________________DEM__________________________________
# #   // https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m#bands
  DEM = ee.Image('USGS/3DEP/10m').clip(myPolygon)

  # maxReducer = ee.Reducer.max()
  # minReducer = ee.Reducer.min()
  # MAX = DEM.reduceRegion(maxReducer, myPolygon).get("elevation")
  # Min = DEM.reduceRegion(minReducer, myPolygon).get("elevation")
  # mapid = DEM.getMapId()
  # map = folium.Map(location=[yMin.getInfo() ,xMin.getInfo()], zoom_start=12)
  # folium.TileLayer(
  #     tiles=mapid['tile_fetcher'].url_format,
  #     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  #     overlay=True,
  #     name='border',
  #   ).add_to(map)
  # vis_params = {
  #   'min': Min.getInfo(),
  #   'max': MAX.getInfo(),
  #   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
  # # Add the elevation model to the map object.
  # map.add_ee_layer(DEM, vis_params, 'DEM')
  # Add a layer control panel to the map.
  # map.add_child(folium.LayerControl())
  task = ee.batch.Export.image.toDrive(image=DEM,
                                     description='DEM_'+str(id),
                                     folder= folderDEM,
                                     scale=10.2,
                                     region=myPolygon,
                                     fileNamePrefix='DEM_'+str(id),
                                     fileFormat='GeoTIFF')
  task.start()

# task = ee.batch.Export.image.toDrive(landsat, 'exportExample', task_config)


#   #// ______________NAIP_____________________________
#   #// https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ
  
  NAIP_SI = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2014-01-01', '2016-12-31')).filter(ee.Filter.bounds(myPolygon)).mean().clip(myPolygon)
  NAIP_Bands = NAIP_SI.bandNames()
  try:
    for i in range(0,4):
      band = NAIP_Bands.get(i)
      NAIP_Image = NAIP_SI.select([band])

      task = ee.batch.Export.image.toDrive(image=NAIP_Image,
                                      description='NAIP_'+band.getInfo()+'_'+str(id),
                                      folder= folderNAIP,
                                      scale=1,
                                      region=myPolygon,
                                      fileNamePrefix='NAIP_'+band.getInfo()+'_'+str(id),
                                      fileFormat='GeoTIFF')
      task.start()
  except:
    ID_errors.append(id);
    pass


while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

# map


Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id: A2YXEQQ2DOM52AQG75E77YWT).
Polling for task (id